## Create test data

Load all needed data

In [1]:
import os
import dipy as dp
import nibabel as nib
import numpy as np
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, reorient_bvecs, GradientTable
from bonndit.shore import ShoreModel, ShoreFit

os.chdir("/home/olivier/Devel/test/samuel/in4michi/")

# Load fractional anisotropy
dti_fa = nib.load("dti_FA.nii.gz")

# Load DTI mask
dti_mask = nib.load("mask.nii.gz")

# Load and adjust tissue segmentation masks
csf_mask = nib.load("fast_pve_0.nii.gz")
gm_mask = nib.load("fast_pve_1.nii.gz")
wm_mask = nib.load("fast_pve_2.nii.gz")

dti_vecs = nib.load("dti_V1.nii.gz")

data = nib.load("data.nii.gz")

bvals, bvecs = read_bvals_bvecs("bvals", "bvecs")
gtab = gradient_table(bvals, bvecs)

/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def vvi_to_numpy_index(vvi_index):
    new_ind = data.shape[:3] - vvi_index
    return new_ind + (-1,-1,-1)

In [3]:
vvi_to_numpy_index(np.array((95,95,49)))

array([0, 0, 0])

Specify the voxels you want in the test data

In [4]:
wm_voxels = np.array([(49,58,27),(59,33,34),(64,33,32)])
gm_voxels = np.array([(25,44,25),(50,46,25),(55,38,20)])
csf_voxels = np.array([(49,55,25),(38,68,45),(61,28,45)])
add_voxels = np.array([(61,26,23),(60,49,37),(65,24,37)])

TEST_DATA = "/home/olivier/Devel/test/samuel/in4michi/test_data_3d"
if not os.path.exists(TEST_DATA):
    os.mkdir(TEST_DATA)

files = {"dti_FA.nii.gz": dti_fa,
         "mask.nii.gz": dti_mask,
         "fast_pve_0.nii.gz": csf_mask,
         "fast_pve_1.nii.gz": gm_mask,
         "fast_pve_2.nii.gz": wm_mask}

In [5]:
wm_voxels = np.array([tuple(vvi_to_numpy_index(np.array(x))) for x in wm_voxels])

Build the test data

In [6]:
def build_test_data_3d(files):
    for name, img in files.items():
        wm_regions = []
        for voxel in wm_voxels:
            wm_regions.append(np.stack([img.get_data()[tuple(voxel+x)] for x in [(0,0,-1), (0,0,0), (0,0,1)]], axis=0))
        wm_stacked = np.stack(wm_regions, 1)
        
        gm_regions = []
        for voxel in gm_voxels:
            gm_regions.append(np.stack([img.get_data()[tuple(voxel+x)] for x in [(0,0,0), (0,0,0), (0,0,0)]], axis=0))
        gm_stacked = np.stack(gm_regions, 1)
            
        csf_regions = []
        for voxel in csf_voxels:
            csf_regions.append(np.stack([img.get_data()[tuple(voxel+x)] for x in [(0,0,0), (0,0,0), (0,0,0)]], axis=0))
        csf_stacked = np.stack(csf_regions, 1)
        
        complete = np.stack([wm_stacked, gm_stacked, csf_stacked], axis=2)
        print(name, complete.shape)
        img_data = nib.Nifti1Image(complete, img.affine)
        nib.save(img_data, os.path.join(TEST_DATA, name))        

In [7]:
build_test_data_3d(files)
build_test_data_3d({"dti_V1.nii.gz": dti_vecs})
build_test_data_3d({"data.nii.gz": data})

fast_pve_2.nii.gz (3, 3, 3)
fast_pve_1.nii.gz (3, 3, 3)
dti_FA.nii.gz (3, 3, 3)
mask.nii.gz (3, 3, 3)
fast_pve_0.nii.gz (3, 3, 3)
dti_V1.nii.gz (3, 3, 3, 3)
data.nii.gz (3, 3, 3, 97)


## Calculate response result for test data with old code

In [8]:
from subprocess import check_output
from os.path import expanduser
import os

In [9]:
home = expanduser("~")
DATA_DIR = os.path.join(home, 'Devel/test/samuel/in4michi/test_data_3d')
OUT_DIR = os.path.join(DATA_DIR, 'results')

In [10]:
SCRIPT = 'shore-response.py '
venv = '. ' + os.path.join(home, 'Devel/venvs/bonndit/bin/activate') + ' && '

command = venv + SCRIPT + '{dire}/ {out}/response.npz --mask \
{dire}/mask.nii.gz'.format(dire=DATA_DIR, out = OUT_DIR)
print(command)
out = check_output(command, shell = True)

for line in out.splitlines():
    print(line.decode("utf-8") )

. /home/olivier/Devel/venvs/bonndit/bin/activate && shore-response.py /home/olivier/Devel/test/samuel/in4michi/test_data_3d/ /home/olivier/Devel/test/samuel/in4michi/test_data_3d/results/response.npz --mask /home/olivier/Devel/test/samuel/in4michi/test_data_3d/mask.nii.gz
order (radial / angular): 4 / 4
Using provided DTI mask.
loading data....
csf response:
  0.   done   
3 voxel
[ 91630.14341921 -90293.49816839 198821.42034613]
grey matter response:
  0.   done   
5 voxel
[87092.07357143 -6570.3734587   9918.82517223]
white matter response:
  0.   done   
0 voxel
[0. 0. 0. 0. 0. 0.]


## Calculate deconvolution results for test data with old code

In [11]:
SCRIPT = 'shore-deconvolve.py '
OPTIONS = ['hpsd', 'nonneg', 'none']
venv = '. ' + os.path.join(home, 'Devel/venvs/bonndit/bin/activate') + ' && '

for option in OPTIONS:
    command = venv + SCRIPT + '{dire}/data.nii.gz {dire}/response_complete.npz \
{out}/odf{opt}.nrrd --pos {opt} --csf {out}/csf{opt}.nrrd \
--gm {out}/gm{opt}.nrrd --wm {out}/wm{opt}.nrrd'.format(dire=DATA_DIR, opt=option, out=OUT_DIR)
    print(command)
    out = check_output(command, shell = True)
    for line in out.splitlines():
        print(line.decode("utf-8") )

. /home/olivier/Devel/venvs/bonndit/bin/activate && shore-deconvolve.py /home/olivier/Devel/test/samuel/in4michi/test_data_3d/data.nii.gz /home/olivier/Devel/test/samuel/in4michi/test_data_3d/response_complete.npz /home/olivier/Devel/test/samuel/in4michi/test_data_3d/results/odfhpsd.nrrd --pos hpsd --csf /home/olivier/Devel/test/samuel/in4michi/test_data_3d/results/csfhpsd.nrrd --gm /home/olivier/Devel/test/samuel/in4michi/test_data_3d/results/gmhpsd.nrrd --wm /home/olivier/Devel/test/samuel/in4michi/test_data_3d/results/wmhpsd.nrrd
order (radial / angular): 4 / 4
[[281.77762318   0.           0.         ...   0.         455.3333953
  651.25522922]
 [281.77762318   0.           0.         ...   0.         455.3333953
  651.25522922]
 [281.77762318   0.           0.         ...   0.         455.3333953
  651.25522922]
 ...
 [ 93.22590013 -56.69754511  47.41910738 ...   5.52542865  82.49419902
   29.13357967]
 [ 93.22590013 -16.66570777 -28.95119336 ... -35.08859464  82.49419902
   29.13